<a href="https://colab.research.google.com/github/nicks165/VectorDatabases/blob/main/zilliz_wiki_2m.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://zilliz.com/doc/integrate_with_cohere


https://zilliz.com/doc/quick_start

In [1]:
!pip install grpcio<=1.53.0
!pip install pymilvus==2.2.8 cohere datasets
!pip install --upgrade ipython

/bin/bash: =1.53.0: No such file or directory
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Initiatize Zilliz, create collection in code. Database created manually at https://cloud.zilliz.com/projects/MA==/databases

In [2]:
import cohere
from pymilvus import connections, FieldSchema, CollectionSchema, DataType, Collection, utility

URI = " https://in01-8515adca5ddbe2b.aws-us-west-2.vectordb.zillizcloud.com:19543"
USER = "user"
PASSWORD = "deeplake_benchmarks"

# Instantiate the client with the auth config
# Connect to Milvus Database
connections.connect(uri=URI, user=USER, password=PASSWORD, secure=True)

ConnectionConfigException: ignored

Load datset and get the embeddings from Cohere dataset at Huggingface

In [ ]:
from datasets import load_dataset
import torch
import cohere

#Load all documents + embeddings
limit = 1000000 # keep -1 for all, else update to a positive number to limit

max_docs_loaded = 0
docs_stream = load_dataset(f"Cohere/wikipedia-22-12-zh-embeddings", split="train")
docs = []
doc_embeddings = []

for doc in docs_stream:
    docs.append(doc)
    doc_embeddings.append(doc['emb'])
    max_docs_loaded += 1
    if (limit > 0 and max_docs_loaded == limit):
      break

Initialize index/collection (class in zilliz)

In [ ]:
COLLECTION_NAME = "wiki_cohere_2m"
DIMENSION = 768

# Remove collection if it already exists
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

# Create collection which includes the id, title, and embedding.
fields = [
    FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name='text', dtype=DataType.VARCHAR, max_length=20000),
    FieldSchema(name='vector', dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)

# Create an AutoIndex index for collection.
index_params = {
    'index_type': 'AUTOINDEX',
    'metric_type': 'IP',
    'params':{"nlist":1024}
}

## Creatig a idex is extremely slow. Hamgs the batch imsert
collection.create_index(field_name="vector", index_params=index_params)
collection.load()

Confirm shape and save the dataset to disk

In [ ]:
import numpy as np

shape = np.array(doc_embeddings).shape
print(shape)

#docs_stream.save_to_disk("cohere dataset")

Load embeddings and metadata into Zilliz

In [ ]:
import time

batch_size=100

# Configure a batch process
for i in range(0, 10000, batch_size):
  # start timer
  start_time = time.time()

  data = [
        [doc["text"] for doc in docs[i:i+batch_size]],
        doc_embeddings[i:i+batch_size]      
    ]

  collection.insert(data)

  # i+1 because index starts at 0 :)
  if(i+1 in [shape[0], batch_size, 1000, 10000, 100000, 500000, 1000000, 2000000]):
    print(" For {0} entries, time taken for inserts = {1} ".format(i+1, time.time() - start_time))

collection.flush()


Querying the dataset

In [4]:
import json

co = cohere.Client(f"o7lTEJeC1QHjU5I4Ee6U2I0m6l5wCOUPWqwoGM7H")  # Add your cohere API key from www.cohere.com

query1 = "What was the cause of the major recession in the early 20th century?"
query2 = "Where is Mount Everest?"
query3 = ""

# create the query embedding
nearVector = co.embed(texts=[query1], model='multilingual-22-12').embeddings

query_start_time = time.time()

results = collection.search(
        data = nearVector,  # Embeded the question
        anns_field='vector',  # Search across the text embeddings
        param={},
        limit = 5,  # Limit to top_k results per search
        output_fields=['text']  # Include the text in the result
    )

ret = []
for hit in results[0]:
  row = []
  row.extend([hit.entity.get('text'), hit.score])  # Get the original text, distance for the results
  ret.append(row)

print(" For 1 query, time taken for search = {0} ".format(time.time() - query_start_time))

print(json.dumps(ret, indent=4))

NameError: ignored